In [2]:
from ultralytics import YOLO
import cv2

model = YOLO(r"E:\waste_detection\best_new.pt")
results = model.predict(source="0", show=True)
print(results)


1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 608x800 2 Plastics, 291.4ms
0: 608x800 3 Plastics, 251.3ms
0: 608x800 (no detections), 247.9ms
0: 608x800 3 Plastics, 234.6ms
0: 608x800 1 Bio, 232.7ms
0: 608x800 2 Bios, 240.6ms
0: 608x800 1 Bio, 215.5ms
0: 608x800 2 Bios, 1 Plastic, 221.6ms
0: 608x800 1 Bio, 1 Plastic, 237.3ms
0: 608x800 1 Bio, 220.4ms
0: 608x800 1 Bio, 1 Plastic, 238.5ms
0: 608x800 1 Bio, 233.3ms
0: 608x800 1 Bio, 231.8

KeyboardInterrupt: 

: 

In [ ]:
import cv2
import torch
import requests
import json
from collections import deque
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    data = command + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.137.20/data'

# URL for the stream
stream_url = "http://192.168.137.195:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Variables to track labels
labels = deque(maxlen=5)  # Store labels for the last 5 frames
consistent_labels = 0  # Counter to track consistent labels

# Continuously process frames and send labels to ESP32
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    cv2.imshow("Frame", resized_frame)
    results = model(resized_frame)

    # Store labels for the current frame
    current_labels = []

    for pred in results:  # Iterate over the results directly
        # Retrieve the label with the highest confidence
        if pred.boxes.conf is not None and len(pred.boxes) > 0:
            max_conf_idx = torch.argmax(pred.boxes.conf)
            label_id = int(pred.boxes.cls[max_conf_idx])
            class_names = {0: 'B', 1: 'P', 2: 'm'}
            label = class_names.get(label_id, 'P')  # Default label is 'P'

            # Append the detected label to the list of current labels
            current_labels.append(label)

    # Store labels in the deque
    labels.append(current_labels)

    # Check if labels have been consistent for 5 frames
    if len(labels) == labels.maxlen and len(set(labels)) == 1:
        consistent_labels += 1
        if consistent_labels == 5:
            consistent_labels = 0  # Reset the counter
            # Send the label to ESP32
            label_to_send = labels[0][0]  # Send the label from the first frame
            send_command(label_to_send)

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch
import requests
from collections import deque
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    data = command + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.137.23/data'

# URL for the stream
stream_url = "http://192.168.137.170:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Variables to track labels
labels = deque(maxlen=5)  # Store labels for the last 5 frames
consistent_labels = 0  # Counter to track consistent labels

# Continuously process frames and send labels to ESP32
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model(resized_frame)

    # Store labels for the current frame
    current_labels = []

    for pred in results:  # Iterate over the results directly
        # Retrieve the label with the highest confidence
        if pred.boxes.conf is not None and len(pred.boxes) > 0:
            max_conf_idx = torch.argmax(pred.boxes.conf)
            label_id = int(pred.boxes.cls[max_conf_idx])
            class_names = {0: 'B', 1: 'G', 2: 'M', 3: 'P'}
            label = class_names.get(label_id, 'P')  # Default label is 'P'

            # Append the detected label to the list of current labels
            current_labels.append(label)

    # Store labels in the deque as a tuple
    labels.append(tuple(current_labels))

    # Check if labels have been consistent for 5 frames
    if len(labels) == labels.maxlen and len(set(labels)) == 1:
        consistent_labels += 1
        if consistent_labels == 5:
            consistent_labels = 0  # Reset the counter
            # Send the label to ESP32
            label_to_send = labels[0][0]  # Send the label from the first frame
            send_command(label_to_send)

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch
import requests
from collections import deque
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    data = command + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.137.23/data'

# URL for the stream
stream_url = "http://192.168.137.170:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Variables to track labels
labels = deque(maxlen=5)  # Store labels for the last 5 frames
consistent_labels = 0  # Counter to track consistent labels

# Continuously process frames and send labels to ESP32
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model(resized_frame)

    # Store labels for the current frame
    current_labels = []

    for pred in results:  # Iterate over the results directly
        if hasattr(pred.boxes, 'cls') and len(pred.boxes) > 0:
            max_conf_idx = torch.argmax(pred.boxes.conf)
            label_id = int(pred.boxes.cls[max_conf_idx])
            class_names = {0: 'B', 1: 'G', 2: 'M', 3: 'P'}
            label = class_names.get(label_id, 'P')  # Default label is 'P'
            current_labels.append(label)

    # Store labels in the deque as a tuple
    labels.append(tuple(current_labels))

    # Check if labels have been consistent for 5 frames
    if len(labels) == labels.maxlen and len(set(labels)) == 1:
        consistent_labels += 1
        if consistent_labels == 5:
            consistent_labels = 0  # Reset the counter
            # Send the label to ESP32
            if labels and labels[0]:
                label_to_send = labels[0][0]  # Send the label from the first frame
                send_command(label_to_send)

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import requests
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    # Map categories to specific identifiers recognized by the ESP32
    category_to_command = {
        'Plastic': 'P',
        'Biodegradable': 'B',
        'Metal': 'M',
        'Glass': 'O'  # Change 'Glass' to 'O' for the ESP32
    }

    # Check if the predicted category is in the mapping, if not, default to 'P' (Plastic)
    cmd = category_to_command.get(command, 'P')
    
    data = cmd + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.137.35/data'

# URL for the stream
stream_url = "http://192.168.137.170:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Continuously process frames and send labels to ESP32
while True:
    ret, frame = cap.read()
    if not ret:
        break

    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model(resized_frame)

    for result in results:
        if result.boxes.tensor.shape[0] > 0:  # Checking if there are detections
            for box in result.boxes.tensor:
                label_id = int(box[5])  # Accessing the class ID

                class_names = {0: 'Biodegradable', 1: 'Glass', 2: 'Metal', 3: 'Plastic'}
                label = class_names.get(label_id, 'Plastic')  # Default label is 'Plastic'

                print("Label sent to ESP32:", label)

    # Display the frame here

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import torch
import requests
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    data = command + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.137.35/data'

# URL for the stream
stream_url = "http://192.168.137.170:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Continuously process frames and send labels to ESP32
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model(resized_frame)

    # Process each prediction in the results list
    for pred in results:  # Iterate over the results directly
        print("Prediction:", pred)  # Print the prediction to understand its structure

        # Access prediction elements to assign labels (replace this with correct indices)
        if len(pred) > 5:  # Ensure it has the required length for label indexing
            label_id = pred[5]  # Update the index based on the actual structure
            label = 'P'  # Default label is 'P' (plastic) for demonstration purposes

            # Logic to assign labels based on label IDs
            if label_id == 0:
                label = 'B'  # Biodegradable
            elif label_id == 1:
                label = 'G'  # Glass
            elif label_id == 2:
                label = 'M'  # Metal
            elif label_id == 3:
                label = 'P'  # Plastic

            # Print the label before sending to ensure correctness
            print("Label sent to ESP32:", label)

            # Send label to ESP32
            send_command(label)

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import requests
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    url = 'http://192.168.137.35/data'
    data = command + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# URL for the stream
stream_url = "http://192.168.137.170:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")
    exit()

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Default label is 'X' for demonstration purposes
default_label = 'P'

# Read and display the video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error reading frame")
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)

    max_confidence_label = default_label
    max_confidence = 0.0

    for result in results:
        for prediction in result:
            if len(prediction) >= 6:  # Check the minimum length required
                label_id = prediction[5]  # Accessing the class ID
                confidence = prediction[4]  # Accessing the confidence value

                # Logic to assign labels based on label IDs
                if confidence > max_confidence:
                    if label_id == 1:
                        max_confidence_label = 'O'
                    elif label_id == 0:
                        max_confidence_label = 'B'
                    elif label_id == 2:
                        max_confidence_label = 'M'
                    elif label_id == 3:
                        max_confidence_label = 'P'
                    max_confidence = confidence

    # Print the label with maximum confidence before sending to ensure correctness
    print("Label sent to ESP32:", max_confidence_label)

    # Send label with maximum confidence to ESP32
    send_command(max_confidence_label)

    # Display the original frame
    cv2.imshow('Frame', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch
import requests
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    data = command + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.137.35/data'

# URL for the stream
stream_url = "http://192.168.137.170:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Initialize label outside the loop
label = 'P'  # Default label is 'P' (plastic) for demonstration purposes

# Continuously process frames and send labels to ESP32
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)

    for result in results:
        for prediction in result:
            # Check if the prediction is not an empty list and contains necessary elements
            if len(prediction) >= 6:  # Check the minimum length required
                label_id = prediction[5]  # Accessing the class ID

                # Logic to assign labels based on label IDs
                label = 'P'  # Default label is 'P' (plastic) for demonstration purposes
                if label_id == 1:
                    label = 'O'
                elif label_id == 0:
                    label = 'B'
                elif label_id == 2:
                    label = 'M'
                elif label_id == 3:
                    label = 'P'

                # Print the label before sending to ensure correctness
                print("Label sent to ESP32:", label)

        # Send label to ESP32
        send_command(label)

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import torch
import requests
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    data = command + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.137.35/data'

# URL for the stream
stream_url = "http://192.168.137.170:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Continuously process frames and send labels to ESP32
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)

    for result in results:
        for prediction in result:
            # Check if the prediction is not an empty list and contains necessary elements
            if len(prediction) >= 6:  # Check the minimum length required
                label_id = prediction[5]  # Accessing the class ID

                # Logic to assign labels based on label IDs
                label = 'P'  # Default label is 'P' (plastic) for demonstration purposes
                if label_id == 0:
                    label = 'B'  # Biodegradable
                elif label_id == 1:
                    label = 'G'  # Glass
                elif label_id == 2:
                    label = 'M'  # Metal
                elif label_id == 3:
                    label = 'P'  # Plastic

                # Print the label before sending to ensure correctness
                print("Label sent to ESP32:", label)

                # Send label to ESP32
                send_command(label)

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
from ultralytics import YOLO

# URL for the stream
stream_url = "http://192.168.137.195:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\waste_detection\best_new.pt")

# Read and display the video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640,480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)
    print(results)

    # Display the original frame
    cv2.imshow('Frame', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO
model = YOLO(r"E:\best.pt")
model.export(format="saved_model")

In [ ]:
import cv2

# Initialize the camera
cap = cv2.VideoCapture(0)  # Use 0 for the default camera or specify the camera index

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Display the frame
    cv2.imshow("Camera Stream", frame)

    # Check for a key press and break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Exit the loop when 'q' is pressed

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch
import requests
from collections import deque
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    data = str(command) + '\n'  # Convert the command to a string before sending
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.137.20/data'

# URL for the stream
stream_url = "http://192.168.137.195:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Variables to track labels
labels = deque(maxlen=5)  # Store labels for the last 5 frames
consistent_labels = 0  # Counter to track consistent labels

# Define the class names
class_names = {0: 'Bio', 1: 'Plastic', 2: 'metal'}

# Continuously process frames and send labels to ESP32
while True:
    ret, frame = cap.read()
    if not ret:  # If no frame is read
        label_to_send = 'O'  # Send 'O' as the command
        send_command(label_to_send)
        continue  # Move to the next iteration

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model(resized_frame)

    # Store labels for the current frame
    current_labels = []

    for pred in results:  # Iterate over the results directly
        if hasattr(pred.boxes, 'cls') and len(pred.boxes) > 0:
            max_conf_idx = torch.argmax(pred.boxes.conf)
            label_id = int(pred.boxes.cls[max_conf_idx])
            label = class_names.get(label_id, 'Plastic')  # Default label is 'Plastic'
            current_labels.append(label)

    # Store labels in the deque as a tuple
    labels.append(tuple(current_labels))

    # Check if labels have been consistent for 5 frames
    if len(labels) == labels.maxlen and len(set(labels)) == 1:
        consistent_labels += 1
        if consistent_labels == 5:
            consistent_labels = 0  # Reset the counter
            # Send the label to ESP32
            if labels and labels[0]:
                label_to_send = labels[0][0]  # Send the label from the first frame
                send_command(label_to_send)

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch
import requests
from collections import deque, Counter
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    data = str(command) + '\n'  # Convert the command to a string before sending
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0
    
# ESP IP URL
url = 'http://192.168.137.20/data'

# URL for the stream
stream_url = "http://192.168.137.195:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Variables to track labels
labels = deque(maxlen=5)  # Store labels for the last 5 frames
consistent_labels = 0  # Counter to track consistent labels

# Define the class names
class_names = {0: 'Bio', 1: 'Plastic', 2: 'metal'}

# Continuously process frames and send labels to ESP32
while True:
    ret, frame = cap.read()
    if not ret:  # If no frame is read
        label_to_send = 'O'  # Send 'O' as the command
        send_command(label_to_send)
        continue  # Move to the next iteration

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model(resized_frame)

    # Store labels for the current frame
    current_labels = []

    for pred in results:  # Iterate over the results directly
        if hasattr(pred.boxes, 'cls') and len(pred.boxes) > 0:
            max_conf_idx = torch.argmax(pred.boxes.conf)
            label_id = int(pred.boxes.cls[max_conf_idx])
            label = class_names.get(label_id, 'Plastic')  # Default label is 'Plastic'
            current_labels.append(label)

    # Store labels in the deque as a tuple
    labels.append(tuple(current_labels))

    # Check if labels have been consistent for 5 frames
    if len(labels) == labels.maxlen:
        counter = Counter(labels)
        most_common = counter.most_common(1)  # Get the most common label and its count
        consistent_labels += 1
        if consistent_labels == 5:
            consistent_labels = 0  # Reset the counter
            # Send the label to ESP32
            if most_common:
                label_to_send = most_common[0][0] if most_common[0][1] > 1 else 'O'  # Send the most common label, or 'O' if not consistent
                send_command(label_to_send)

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import torch
import requests
from collections import deque
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    data = str(command) + '\n'  # Convert the command to a string before sending
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0
# (remaining function code remains unchanged)

# ESP IP URL
url = 'http://192.168.137.157/data'

# URL for the stream
stream_url = "http://192.168.137.195:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Variables to track labels
labels = deque(maxlen=5)  # Store labels for the last 5 frames
consistent_labels = 0  # Counter to track consistent labels

# Continuously process frames and send labels to ESP32
while True:
    ret, frame = cap.read()
    if not ret:
        cap = cv2.VideoCapture(stream_url)  # Re-open the stream if it ends
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model(resized_frame)

    # Store labels for the current frame
    current_labels = []

    for pred in results:
        if pred.boxes.conf is not None and len(pred.boxes) > 0:
            max_conf_idx = torch.argmax(pred.boxes.conf)
            label_id = int(pred.boxes.cls[max_conf_idx])
            class_names = {0: 'B', 1: 'P', 2: 'm'}
            label = class_names.get(label_id, 'P')  # Default label is 'P'
            current_labels.append(label)

    # Convert the list of labels to a tuple before storing
    labels.append(tuple(current_labels))

    # Check if labels have been consistent for 5 frames
    # Send label if consistent for 5 frames
    if len(labels) == labels.maxlen and len(set(labels)) == 1 and labels and labels[0]:
        consistent_labels += 1
        if consistent_labels == 5:
            consistent_labels = 0  # Reset the counter
            # Send the label to ESP32
            label_to_send = labels[0][0]  # Send the label from the first frame
            send_command(label_to_send)

    # Print labels for debugging purposes
    print("Labels:", labels)

    # Keep the serial output continuous by handling exceptions
    try:
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except Exception as e:
        print("Exception occurred:", e)
        continue

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()



0: 608x800 1 Plastic, 270.7ms
Speed: 0.0ms preprocess, 270.7ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)



Labels: deque([('P',)], maxlen=5)


0: 608x800 (no detections), 238.3ms
Speed: 5.0ms preprocess, 238.3ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)



Labels: deque([('P',), ()], maxlen=5)


0: 608x800 2 Plastics, 213.3ms
Speed: 19.0ms preprocess, 213.3ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)



...

0: 608x800 1 Glass, 1 Plastic, 272.5ms
Speed: 0.0ms preprocess, 272.5ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)



Labels: deque([('B',), ('B',), ('B',), ('P',), ('P',)], maxlen=5)


0: 608x800 1 Plastic, 280.6ms
Speed: 14.7ms preprocess, 280.6ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)



Labels: deque([('B',), ('B',), ('P',), ('P',), ('P',)], maxlen=5)


KeyboardInterrupt: 